In [43]:
import pandas as pd
import datetime
import numpy
import re
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
from sklearn.ensemble import RandomForestClassifier

In [44]:
# Function for calculating age of a person given his date of birth
from datetime import date
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [45]:
#Function for calculating number of days between two dates 
def datediff(any_date):
    today = date.today()
    datediff = today-any_date 
    return datediff.days

In [46]:
# Calculating the Tenure in months given the data is in X Years and Y months format
def age_length(TenureAsYearsAndMonths):
    years = int(re.findall(r'(\d+)yrs',TenureAsYearsAndMonths )[0])
    months = int(re.findall(r'(\d+)mon',TenureAsYearsAndMonths )[0])
    total_tenure = (years*12)+months
    return total_tenure

In [47]:
# Function for doing all the data manipulations
def AllDataManipulations(MainDataFrame,Manip_Data = pd.DataFrame,temp1 = pd.DataFrame, temp2 = pd.DataFrame,Manip_Completed = pd.DataFrame):
    Manip_Data = MainDataFrame.copy()
    Manip_Data['Employment.Type'].fillna(Manip_Data['Employment.Type'].mode()[0],inplace = True)
    Manip_Data[['loan_default','branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']] = Manip_Data[
        ['loan_default','branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']].apply(lambda x: x.astype('category'))
    Manip_Data['Date.of.Birth'] = pd.to_datetime(Manip_Data['Date.of.Birth'])
    Manip_Data['Age'] = Manip_Data['Date.of.Birth'].apply(lambda x: calculate_age(x))
    Manip_Data['DisbursalDate'] = pd.to_datetime(Manip_Data['DisbursalDate'])
    Manip_Data['HowManyDaysSinceDisburse'] = Manip_Data['DisbursalDate'].dt.date.apply(lambda x: datediff(x))
    Manip_Data['AvgAcctAgeInMonths'] = Manip_Data['AVERAGE.ACCT.AGE'].apply(lambda x: age_length(x))
    Manip_Data['CredHistLenInMonts'] = Manip_Data['CREDIT.HISTORY.LENGTH'].apply(lambda x: age_length(x))
    temp1 = pd.get_dummies(Manip_Data['Employment.Type'],prefix='EmploymentType')
    temp2 = pd.get_dummies(Manip_Data['PERFORM_CNS.SCORE.DESCRIPTION'],prefix='Bureau_score_description')
    Manip_Completed = pd.concat([Manip_Data[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID','Age',
       'HowManyDaysSinceDisburse','State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AvgAcctAgeInMonths','CredHistLenInMonts','NO.OF_INQUIRIES',
       'loan_default']],temp1,temp2],axis=1)
    return Manip_Data,Manip_Completed

In [48]:
MasterData = pd.read_csv("train.csv")
Data1 = MasterData.copy()

In [49]:
Train_ManipData_1,Train_ManipCompleted_1 = AllDataManipulations(Data1)

In [50]:
Train_ManipCompleted_1.columns

Index(['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 'loan_default',
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',

In [51]:
IndeVectors = Train_ManipCompleted_1[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',
       'Bureau_score_description_C-Very Low Risk',
       'Bureau_score_description_D-Very Low Risk',
       'Bureau_score_description_E-Low Risk',
       'Bureau_score_description_F-Low Risk',
       'Bureau_score_description_G-Low Risk',
       'Bureau_score_description_H-Medium Risk',
       'Bureau_score_description_I-Medium Risk',
       'Bureau_score_description_J-High Risk',
       'Bureau_score_description_K-High Risk',
       'Bureau_score_description_L-Very High Risk',
       'Bureau_score_description_M-Very High Risk',
       'Bureau_score_description_No Bureau History Available',
       'Bureau_score_description_Not Scored: More than 50 active Accounts found',
       'Bureau_score_description_Not Scored: No Activity seen on the customer (Inactive)',
       'Bureau_score_description_Not Scored: No Updates available in last 36 months',
       'Bureau_score_description_Not Scored: Not Enough Info available on the customer',
       'Bureau_score_description_Not Scored: Only a Guarantor',
       'Bureau_score_description_Not Scored: Sufficient History Not Available']]
Labels = Train_ManipCompleted_1['loan_default']

In [52]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(IndeVectors,Labels,random_state = 1234,test_size = 0.10)

In [53]:
print(len(train_x))
print(len(train_y))
print(len(valid_x))
print(len(valid_y))

209838
209838
23316
23316


In [54]:
# Random forest
rf_model_1 = RandomForestClassifier(n_estimators=300,bootstrap = True,max_features = 'sqrt')
rf_model_1.fit(train_x,list(train_y))
ypred_rf1 = rf_model_1.predict(valid_x)
auc_rf1 = metrics.roc_auc_score(valid_y,ypred_rf1)
print("AUC of Random forest Model_1: ",auc_rf1)
# Naive Bayes
gnb = GaussianNB()
gnb.fit(train_x,train_y)
ypred_gnb = gnb.predict(valid_x)
auc_gnb = metrics.roc_auc_score(valid_y,ypred_gnb)
print("AUC of Naive Bayes Model_1: ",auc_gnb)

AUC of Random forest Model_1:  0.5140930876195822
AUC of Naive Bayes Model_1:  0.4977692566765253


In [28]:
# Modifying data manipulations function to work for test data as it doesn't have labels column
# Function for doing all the data manipulations
def AllDataManipulations_forTest(MainDataFrame,Manip_Data = pd.DataFrame,temp1 = pd.DataFrame, temp2 = pd.DataFrame,Manip_Completed = pd.DataFrame):
    Manip_Data = MainDataFrame.copy()
    Manip_Data['Employment.Type'].fillna(Manip_Data['Employment.Type'].mode()[0],inplace = True)
    Manip_Data[['branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']] = Manip_Data[
        ['branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']].apply(lambda x: x.astype('category'))
    Manip_Data['Date.of.Birth'] = pd.to_datetime(Manip_Data['Date.of.Birth'])
    Manip_Data['Age'] = Manip_Data['Date.of.Birth'].apply(lambda x: calculate_age(x))
    Manip_Data['DisbursalDate'] = pd.to_datetime(Manip_Data['DisbursalDate'])
    Manip_Data['HowManyDaysSinceDisburse'] = Manip_Data['DisbursalDate'].dt.date.apply(lambda x: datediff(x))
    Manip_Data['AvgAcctAgeInMonths'] = Manip_Data['AVERAGE.ACCT.AGE'].apply(lambda x: age_length(x))
    Manip_Data['CredHistLenInMonts'] = Manip_Data['CREDIT.HISTORY.LENGTH'].apply(lambda x: age_length(x))
    temp1 = pd.get_dummies(Manip_Data['Employment.Type'],prefix='EmploymentType')
    temp2 = pd.get_dummies(Manip_Data['PERFORM_CNS.SCORE.DESCRIPTION'],prefix='Bureau_score_description')
    Manip_Completed = pd.concat([Manip_Data[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID','Age',
       'HowManyDaysSinceDisburse','State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AvgAcctAgeInMonths','CredHistLenInMonts','NO.OF_INQUIRIES']],temp1,temp2],axis=1)
    return Manip_Data,Manip_Completed

In [29]:
Master_test = pd.read_csv("test_bqCt9Pv.csv")
Test_Data = Master_test.copy()
Test_ManipData,Test_ManipCompleted = AllDataManipulations_forTest(Test_Data)

In [30]:
Test_Data.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
0,655269,53478,63558,86.54,67,22807,45,1497,01-01-74,Salaried,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
1,723482,55513,63163,89.45,67,22807,45,1497,20-05-85,Self employed,...,0,0,0,5605,0,1,0,0yrs 8mon,1yrs 0mon,1
2,758529,65282,84320,79.93,78,23135,86,2071,14-10-95,Salaried,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
3,763449,46905,63896,76.58,78,17014,45,2070,01-06-73,Self employed,...,0,0,0,0,0,0,0,2yrs 5mon,2yrs 5mon,0
4,708663,51428,63896,86.08,78,17014,45,2069,01-06-72,Salaried,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0


In [37]:
Test_ManipCompleted.columns

Index(['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 'EmploymentType_Salaried',
       'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',
       'Bureau_

In [35]:
Test_ManipCompleted['Bureau_score_description_Not Scored: More than 50 active Accounts found'] = 0

In [36]:
Test_ManipCompleted['Bureau_score_description_Not Scored: More than 50 active Accounts found'].unique()

array([0], dtype=int64)

In [64]:
ActualTestPred_rf1 = rf_model_1.predict(Test_ManipCompleted)
ActualTestPred_nb = gnb.predict(Test_ManipCompleted)
RF1_Submissions = pd.concat([Test_Data['UniqueID'],pd.DataFrame(ActualTestPred_rf1,columns=['loan_default'])],axis=1)
nb_Submissions = pd.concat([Test_Data['UniqueID'],pd.DataFrame(ActualTestPred_nb,columns=['loan_default'])],axis=1)

In [65]:
RF1_Submissions.to_csv('RF1_Submissions.csv',index=False)

In [41]:
# Finding Feature importance
RF_feature_importances = pd.DataFrame(rf_model_1.feature_importances_,
                                   index = train_x.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

In [ ]:
# Trying out more random forests with more number of trees

# Second model with 600 trees, tweaking the max_features as well 
rf_model_2 = RandomForestClassifier(n_estimators=600,bootstrap = False,max_features = 59,random_state=1234)
rf_model_2.fit(train_x,list(train_y))
ypred_rf2 = rf_model_1.predict(valid_x)
auc_rf2 = metrics.roc_auc_score(valid_y,ypred_rf2)
print("AUC of Random forest Model_2: ",auc_rf2)

# Third model with 1000 trees, tweaking the max_features as well 
rf_model_3 = RandomForestClassifier(n_estimators=1000,bootstrap = False,max_features = 59,random_state=1234)
rf_model_3.fit(train_x,list(train_y))
ypred_rf3 = rf_model_1.predict(valid_x)
auc_rf3 = metrics.roc_auc_score(valid_y,ypred_rf3)
print("AUC of Random forest Model_3: ",auc_rf3)

# Fourth model with 600 trees, tweaking the max_features to sqrt 
rf_model_4 = RandomForestClassifier(n_estimators=600,bootstrap = False,max_features = 'sqrt',random_state=1234)
rf_model_4.fit(train_x,list(train_y))
ypred_rf4 = rf_model_1.predict(valid_x)
auc_rf4 = metrics.roc_auc_score(valid_y,ypred_rf4)
print("AUC of Random forest Model_4: ",auc_rf4)

# Fifth model with 1000 trees, tweaking the max_features to sqrt 
rf_model_5 = RandomForestClassifier(n_estimators=1000,bootstrap = False,max_features = 'sqrt',random_state=1234)
rf_model_5.fit(train_x,list(train_y))
ypred_rf5 = rf_model_1.predict(valid_x)
auc_rf5 = metrics.roc_auc_score(valid_y,ypred_rf5)
print("AUC of Random forest Model_5: ",auc_rf5)

AUC of Random forest Model_2:  0.5140930876195822
